Figure 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from nbodykit.lab import *
from nbodykit import setup_logging, style
from nbodykit.cosmology import Planck15
from scipy.interpolate import InterpolatedUnivariateSpline as ius
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # many annoying dask warnings from bigfile
plt.style.use(style.notebook)
setup_logging()

Measure $b_{\phi}$ from SU in mass bins via FD in $\sigma_{8}$ and $D(z)$
(Also eventually do concentration splits)

Select fiducial and FD step values of $z$, $\sigma_{8}$

In [ ]:
z0,s80 = 1.0,0.834
s8m = 8.1400e-01
s8p = 8.5400e-01
zstep = 0.05
zm,zp = z0+zstep,z0-zstep

In [ ]:
fid_path = "./"
seed=3
f0 = BigFileCatalog(fid_path+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed))
fm = BigFileCatalog(fid_path+'output_box_2000_fnl_0_s8_minus_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed))
fp = BigFileCatalog(fid_path+'output_box_2000_fnl_0_s8_plus_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed))

fzm = BigFileCatalog(fid_path+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+zm),seed))
fzp = BigFileCatalog(fid_path+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+zp),seed))

In [ ]:
ns = 0.9624# from params file
cosmom = cosmology.Cosmology(h=fm.attrs['HubbleParam'], Omega0_b=0.0,Omega0_cdm=fm.attrs['OmegaM'], n_s=ns, 
                             sigma8=s8m)
s8s = (s8m,s80,s8p)
Dzs = (fzm.attrs['GrowthFactor'],f0.attrs['GrowthFactor'],fzp.attrs['GrowthFactor'])

Load FoF Catalogs

In [ ]:
path_pm = './'
prefix_pm='output_box_2000_fnl_0_s8_plus_s_12'
path_0 = '/pscratch/sd/j/jsull/cori_cscratch/fnl/sfi/'
prefix_0 = 'output_box_2000_fnl_0_s_2' 

In [7]:
# Load power
nbars8s = {"m":[],"0":[],"p":[]}
nbarDzs = {"m":[],"0":[],"p":[]}
path_pk = '/pscratch/sd/j/jsull/cori_cscratch/fnl/bphi/data/power/'
for seed in range(3,13):
    nbars8s["m"].append( 1/np.load(path_pk+'h_b1_sn_z{0}_fnl0.0_Ng1024_L2000.0_s{1}minus_s_{1}.npy'.format(z0,seed))[:,-1] )
    nbars8s["0"].append( 1/np.load(path_pk+'h_b1_sn_z{0}_fnl0.0_Ng1024_L2000.0_s{1}.npy'.format(z0,seed))[:,-1] )
    nbars8s["p"].append( 1/np.load(path_pk+'h_b1_sn_z{0}_fnl0.0_Ng1024_L2000.0_s{1}plus_s_{1}.npy'.format(z0,seed))[:,-1] )

    nbarDzs["m"].append( 1/np.load(path_pk+'h_b1_sn_z{0}_fnl0.0_Ng1024_L2000.0_s{1}.npy'.format(zm,seed))[:,-1] )
    nbarDzs["p"].append( 1/np.load(path_pk+'h_b1_sn_z{0}_fnl0.0_Ng1024_L2000.0_s{1}.npy'.format(zp,seed))[:,-1] )
nbarDzs["0"] = nbars8s["0"]


In [9]:
def bphi(Ns,Xs): # from TNG file
    #X = sigma8 or Dz
    # Function to compute (centered) FD derivative given some numberdensity tuple (Nm,N0,Np)
    # and corresponding redshift/s8 tuple (zm,z0,zp) in order of decreasing redshift (increasing a)
    # or increasing sigma8
    d1pX1,d1pX2 = Xs[0]-Xs[1],-(Xs[1]-Xs[2])
    reld1pX1,reld1pX2 = d1pX1/(Xs[1]),d1pX2/(Xs[1])
    bphi_hi = 2/reld1pX1 *(Ns[0]/Ns[1] -1) #low on N refers to low g-r, redshifts in dec order
    bphi_lo = 2/reld1pX2 *(Ns[2]/Ns[1] -1) 
    #averaging procedure of Barriera++20
    bphi = (bphi_hi + bphi_lo)/2.0
    return bphi


In [ ]:
bphis8s = {"M1":[],"M2":[],"M3":[]}
bphiDzs = {"M1":[],"M2":[],"M3":[]}
for j in range(3):
    for i in range(len(nbars8s['0'])):
        bphis8s['M{0}'.format(j+1)].append(bphi((nbars8s["m"][i][j],nbars8s["0"][i][j],nbars8s["p"][i][j]),s8s))
        bphiDzs['M{0}'.format(j+1)].append(bphi((nbarDzs["m"][i][j],nbarDzs["0"][i][j],nbarDzs["p"][i][j]),Dzs))

In [11]:
fine_mbins=np.logspace(13.1,14.75,20)

In [12]:
def count_bins(cat,mbins):
    cat['Mass'] = cat['Length']*cat.attrs['M0'][0]*1e10
    counts = []
    for i in range(len(fine_mbins)-1):
        counts.append( len(cat['Mass'][(cat['Mass']>mbins[i]) & (cat['Mass']<=mbins[i+1])]) )
    return np.array(counts)

In [ ]:
# Load power
fine_nbars8s = {"m":[],"0":[],"p":[]}
fine_nbarDzs = {"m":[],"0":[],"p":[]}
path_hcat = './'
for seed in range(3,13):
    print("seed: ",seed)
    fine_nbars8s["m"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_0_s8_minus_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbars8s["0"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbars8s["p"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_0_s8_plus_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbarDzs["m"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+zm),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbarDzs["p"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_0_s_{1}/fof_{0:.4f}/'.format(1/(1+zp),seed)+'LL-0.200/'), fine_mbins) )
fine_nbarDzs["0"] = fine_nbars8s["0"]


In [ ]:
# Load power
fine_nbarDzs_fnl100 = {"m":[],"0":[],"p":[]}
path_hcat = '/pscratch/sd/j/jsull/cori_cscratch/fnl/sfi/'
path_hcat = './'
for seed in range(2,11): 
    print("seed: ",seed)
    fine_nbarDzs_fnl100["0"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_100_s_{1}/fof_{0:.4f}/'.format(1/(1+z0),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbarDzs_fnl100["m"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_100_s_{1}/fof_{0:.4f}/'.format(1/(1+zm),seed)+'LL-0.200/'), fine_mbins) )
    fine_nbarDzs_fnl100["p"].append( count_bins(BigFileCatalog(path_hcat+'output_box_2000_fnl_100_s_{1}/fof_{0:.4f}/'.format(1/(1+zp),seed)+'LL-0.200/'), fine_mbins) )


In [58]:
fine_bphis8s = dict(zip(["M{0}".format(i+1) for i in range(len(fine_mbins)-1)],[[] for i in range(len(fine_mbins)-1)]))
fine_bphiDzs = dict(zip(["M{0}".format(i+1) for i in range(len(fine_mbins)-1)],[[] for i in range(len(fine_mbins)-1)]))
for j in range(len(fine_mbins)-1):
    for i in range(len(fine_nbars8s['0'])):

        fine_bphis8s['M{0}'.format(j+1)].append(bphi((fine_nbars8s["m"][i][j],fine_nbars8s["0"][i][j],fine_nbars8s["p"][i][j]),s8s))
        fine_bphiDzs['M{0}'.format(j+1)].append(bphi((fine_nbarDzs["m"][i][j],fine_nbarDzs["0"][i][j],fine_nbarDzs["p"][i][j]),Dzs))

In [59]:
fine_bphiDzs_fnl100 = dict(zip(["M{0}".format(i+1) for i in range(len(fine_mbins)-1)],[[] for i in range(len(fine_mbins)-1)]))
for j in range(len(fine_mbins)-1):
    for i in range(len(fine_nbarDzs_fnl100['0'])):
        fine_bphiDzs_fnl100['M{0}'.format(j+1)].append(bphi((fine_nbarDzs_fnl100["m"][i][j],fine_nbarDzs_fnl100["0"][i][j],fine_nbarDzs_fnl100["p"][i][j]),Dzs))

In [ ]:
import matplotlib.pylab as pl
import matplotlib as mpl
colors = pl.cm.Blues(np.linspace(0,1,len(fine_mbins)-1))
colors2 = pl.cm.Reds(np.linspace(0,1,len(fine_mbins)-1))

cmap = mpl.cm.Blues
norm = mpl.colors.Normalize(vmin=np.log10(fine_mbins.min()), vmax=np.log10(fine_mbins.max()))


In [ ]:
for i in range(len(fine_mbins)-1): plt.scatter(fine_bphis8s['M{0}'.format(i+1)],fine_bphiDzs['M{0}'.format(i+1)])
plt.plot(np.linspace(5,30),np.linspace(5,30),ls='--',c='k')
plt.show()


for i in range(len(fine_mbins)-1):
    plt.scatter(x=np.mean(fine_bphis8s['M{0}'.format(i+1)]),y=np.mean(fine_bphiDzs['M{0}'.format(i+1)]),
                 c=colors[i],
                 marker='.')

plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Blues'),
              orientation='vertical', label=r'$\log_{10}~ M_{h}$')

for i in range(len(fine_mbins)-1):
    plt.errorbar(x=np.mean(fine_bphis8s['M{0}'.format(i+1)]),y=np.mean(fine_bphiDzs['M{0}'.format(i+1)]),
                 xerr=np.std(fine_bphis8s['M{0}'.format(i+1)]),yerr=np.std(fine_bphiDzs['M{0}'.format(i+1)]),
                 capsize=3,capthick=2,
                 c=colors[i],
                 marker='.')

            
plt.plot(np.linspace(6,26),np.linspace(6,26),ls='--',c='k')
plt.xlabel(r'$b_{\phi}^{\sigma_8}$')
plt.ylabel(r'$b_{\phi}^{D(z)}$')
plt.tight_layout()
plt.savefig('../plots/su_fastpm.pdf')
plt.savefig('../plots/su_fastpm.png')
plt.show()